In [1]:
import pandas as pd
import re

In [2]:
df_sandp500_ECT_merge_without_spilt=pd.read_csv("D:/Script/sandp500_conference_earning_transcript.csv")
df_sandp500_ECT_merge_without_spilt

,title,url,date,ticker,text,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,Presentation,QandA,CompanyParticipants,CallParticipants
0,3M Company (MMM) Q3 2023 Earnings Call Transcript,https://seekingalpha.com/article/4642981-3m-co...,2023-10-24,MMM,3M Company (NYSE:MMM) Q3 2023 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator Ladies and gentlemen, thank you for s...",Operator [Operator Instructions] Our first que...,"['Bruce Jermeland', 'Michael Roman', 'Monish P...","['Scott Davis', 'Andrew Obin', 'Joe Ritchie', ..."
1,3M Company (MMM) Q2 2023 Earnings Call Transcript,https://seekingalpha.com/article/4619621-3m-co...,2023-07-25,MMM,3M Company (NYSE:MMM) Q2 2023 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator Ladies and gentlemen, thank you for s...",Operator [Operator Instructions] Our first que...,"['Bruce Jermeland', 'Mike Roman', 'Kevin Rhode...","['Andrew Obin', 'Andrew Kaplowitz', 'Scott Dav..."
2,3M Company (MMM) Q1 2023 Earnings Call Transcript,https://seekingalpha.com/article/4596435-3m-co...,2023-04-25,MMM,3M Company (NYSE:MMM) Q1 2023 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator Ladies and gentlemen, thank you for s...",Operator [Operator Instructions] Our first que...,"['Bruce Jermeland', 'Mike Roman', 'Monish Pato...","['Andrew Kaplowitz', 'Stephen Tusa', 'Scott Da..."
3,3M Company (MMM) Q4 2022 Earnings Call Transcript,https://seekingalpha.com/article/4571946-3m-co...,2023-01-24,MMM,3M Company (NYSE:MMM) Q4 2022 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator Ladies and gentlemen, thank you for s...",Operator [Operator Instructions] Our first que...,"['Bruce Jermeland', 'Mike Roman', 'Monish Pato...","['Scott Davis', 'Andrew Kaplowitz', 'Andrew Ob..."
4,3M Company (MMM) Q3 2022 Earnings Call Transcript,https://seekingalpha.com/article/4548991-3m-co...,2022-10-25,MMM,3M Company (NYSE:MMM) Q3 2022 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator Ladies and gentlemen, thank you for s...",Operator [Operator Instructions] Our first que...,"['Bruce Jermeland', 'Mike Roman', 'Monish Pato...","['Scott Davis', 'Andrew Obin', 'Joe Ritchie', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10188,"Zoetis, Inc. (ZTS) CEO Juan Ramon Alaix on Q2 ...",https://seekingalpha.com/article/4282234-zoeti...,2019-08-06,ZTS,"Zoetis, Inc. (NYSE:ZTS) Q2 2019 Results Earnin...",ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952,Operator Welcome to the Second Quarter 2019 Fi...,Operator [Operator Instructions]. Our first qu...,"['Steve Frank', 'Ramon Alaix', 'Glenn David']","['Kevin Ellich', 'Jon Block', 'Louise Chen', '..."
10189,"Zoetis, Inc. (ZTS) CEO Juan Ramon Alaix on Q1 ...",https://seekingalpha.com/article/4259182-zoeti...,2019-05-02,ZTS,Zoetis Inc. (NYSE:ZTS) Q1 2019 Earnings Confer...,ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952,Operator Welcome to the First Quarter 2019 Fin...,Operator [Operator Instructions] We'll take ou...,"['Steve Frank', 'Ramon Alaix', 'Glenn David']","['Erin Wright', 'Louise Chen', 'Kevin Ellich',..."
10190,"Zoetis, Inc. (ZTS) CEO Juan Ramón Alaix on Q4 ...",https://seekingalpha.com/article/4241305-zoeti...,2019-02-14,ZTS,"Zoetis, Inc. (NYSE:ZTS) Q4 2018 Earnings Confe...",ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952,Operator Welcome to the Fourth Quarter and Ful...,Operator [Operator Instructions] We’ll take to...,"['Steve Frank', 'Ramón Alaix', 'Glenn David']","['Michael Ryskin', 'Kevin Ellich', 'Erin Wrigh..."
10191,Zoetis (ZTS) Q3 2018 Results - Earnings Call 

In [3]:


'''Spilt Presentation, Q&A, Company Participants, Conference Call Participants'''

def find_marker(text, markers):
    """在文本中找到第一个出现的标记，并返回该标记和它的位置"""
    first_index = len(text)  # 初始化一个极大的索引值作为比较基础
    first_marker = None
    for marker in markers:
        index = text.find(marker)
        if index != -1 and (first_marker is None or index < first_index):
            first_index = index
            first_marker = marker
    return first_marker, first_index

def extract_participants(text, start_markers, end_markers):
    """从可能的多个起始和结束标记中提取参与者列表"""
    start_marker, start_index = find_marker(text, start_markers)
    if start_marker is None:
        return ['No sample']  # 如果没有找到起始标记，直接返回空列表

    start_index += len(start_marker)  # 调整开始位置到标记之后

    end_marker, end_index = find_marker(text[start_index:], end_markers)
    if end_marker is not None:
        end_index += start_index  # 调整结束索引为相对于整个文本的位置
    else:
        end_index = None

    participant_section = text[start_index:end_index if end_index is not None else None]

    # 使用正则表达式查找名字，适应名字中可能包含的点和连字符
    '''20240418這個可以處理超多'''
    pattern = re.compile(r'(\b[\w’\'¡¦.]+(?:\s[A-Z]\w*\.)?\s[A-Z][’\'\w]+)(?=\s(?:[–-]|¡V))')

    participants = [match.group(1).strip() for match in pattern.finditer(participant_section)]
    
    return participants

def split_presentation_QandA(text):
    if pd.notna(text):
        # 提取参与者名单
        company_participants = extract_participants(text, ["Company Participants", "Executives", "Corporate Participants", "Company Representatives"], ["Conference Call Participants", "Analysts"])
        call_participants = extract_participants(text, ["Conference Call Participants", "Analysts"], ["Operator"])

        # 初始化段落
        presentation = []
        QandA = []
        in_presentation = False
        in_QandA = False
        last_participant_line = None

        # 分析文本，提取演示和问答部分
        lines = text.split('\n')
        for i, line in enumerate(lines):
            if '' in line:
                # 标记问答开始
                in_presentation = False
                in_QandA = True
                continue  # 跳过这一行，避免将Q&A标记行加入presentation

            # 当在参与者列表更新最后一个行号
            if call_participants and any(participant in line for participant in call_participants):
                last_participant_line = i
            
            # 从最后一个参与者名单后的下一行开始演示内容
            if last_participant_line is not None and i == last_participant_line + 1:
                in_presentation = True

            # 将行添加到正确的部分
            if in_presentation and not in_QandA:
                presentation.append(line)
            elif in_QandA:
                QandA.append(line)
        
        # 将提取的信息添加到结果中
        return ('\n'.join(presentation), '\n'.join(QandA), company_participants, call_participants)
    else:
        # 如果文本为空，则返回空字符串和空列表
        return "", "", [], []


# 应用函数到 DataFrame 的每行，并创建新列
df_sandp500_ECT_merge_without_spilt[['Presentation', 'QandA', 'CompanyParticipants', 'CallParticipants']] = \
    df_sandp500_ECT_merge_without_spilt['text'].apply(
        lambda txt: pd.Series(split_presentation_QandA(txt))
    )




In [20]:
def find_ceo(text, start_pos, end_pos):
    """检查指定文本区段中是否提到 CEO"""
    middle_text = text[start_pos:end_pos]
    return 'CEO' in middle_text or 'Chief Executive Officer' in middle_text

def extract_and_check_ceo(text, company_participants):
    ceo_name = []  # 存储每对参与者之间是否提及 CEO 的信息
    for i in range(len(company_participants) - 1):
        start_participant = company_participants[i]
        end_participant = company_participants[i + 1]
        try:
            start_index = text.index(start_participant) + len(start_participant)
            end_index = text.index(end_participant, start_index)
            # 检查两者之间的文本是否提到 CEO
            if find_ceo(text, start_index, end_index):
                ceo_name.append(start_participant)

        except ValueError:
            # 如果找不到参与者或其他错误，跳过
            continue

    return ceo_name


def process_data(row):
    text = row['text']
    if pd.isna(text):
        # 如果文本是 NaN，则返回空列表或其他适当的默认值
        return pd.Series([[], []])
    
    # 确保这里访问的是单个文本字符串
    company_participants = extract_participants(text, ["Company Participants", "Executives", "Corporate Participants", "Company Representatives"], ["Conference Call Participants", "Analysts"])
    
    # 执行 extract_and_check_ceo 函数
    ceo_mentions = extract_and_check_ceo(text, company_participants)
    
    # 返回更新的信息
    return pd.Series([company_participants, ceo_mentions])


# 添加新列到 DataFrameA
df_sandp500_ECT_merge_without_spilt[['CompanyParticipants', 'CEO']] = df_sandp500_ECT_merge_without_spilt.apply(process_data, axis=1)

In [21]:
df_sandp500_ECT_merge_without_spilt['CEO']

0        [Michael Roman]
1           [Mike Roman]
2           [Mike Roman]
3           [Mike Roman]
4           [Mike Roman]
              ...       
10188      [Ramon Alaix]
10189      [Ramon Alaix]
10190      [Ramón Alaix]
10191                 []
10192                 []
Name: CEO, Length: 10193, dtype: object

In [22]:
# Check Nan/Missing value
nan_pre_index = df_sandp500_ECT_merge_without_spilt[df_sandp500_ECT_merge_without_spilt['Presentation'].isna()].index
nan_pre_index_list = nan_pre_index.tolist()
print(nan_pre_index_list)

[]


In [27]:
save_data = df_sandp500_ECT_merge_without_spilt[['CEO','url']]
save_data

,CEO,url
0,[Michael Roman],https://seekingalpha.com/article/4642981-3m-co...
1,[Mike Roman],https://seekingalpha.com/article/4619621-3m-co...
2,[Mike Roman],https://seekingalpha.com/article/4596435-3m-co...
3,[Mike Roman],https://seekingalpha.com/article/4571946-3m-co...
4,[Mike Roman],https://seekingalpha.com/article/4548991-3m-co...
...,...,...
10188,[Ramon Alaix],https://seekingalpha.com/article/4282234-zoeti...
10189,[Ramon Alaix],https://seekingalpha.com/article/4259182-zoeti...
10190,[Ramón Alaix],https://seekingalpha.com/article/4241305-zoeti...
10191,[],https://seekingalpha.com/article/4217256-zoeti...


In [28]:
save_data.to_csv("D:/Script/CEO.csv", index=False)

In [23]:
# df_sandp500_ECT_merge_without_spilt['Presentation'][450:500]
df_sandp500_ECT_merge_without_spilt.head(10)

,title,url,date,ticker,text,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,Presentation,QandA,CompanyParticipants,CallParticipants,CEOMentions,CEOname,CEO
0,3M Company (MMM) Q3 2023 Earnings Call Transcript,https://seekingalpha.com/article/4642981-3m-co...,2023-10-24,MMM,3M Company (NYSE:MMM) Q3 2023 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator\nLadies and gentlemen, thank you for ...",Operator\n[Operator Instructions] Our first qu...,"[Bruce Jermeland, Michael Roman, Monish Patola...","[Scott Davis, Andrew Obin, Joe Ritchie, Christ...",[Michael Roman],None,[Michael Roman]
1,3M Company (MMM) Q2 2023 Earnings Call Transcript,https://seekingalpha.com/article/4619621-3m-co...,2023-07-25,MMM,3M Company (NYSE:MMM) Q2 2023 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator\nLadies and gentlemen, thank you for ...",Operator\n[Operator Instructions] Our first qu...,"[Bruce Jermeland, Mike Roman, Kevin Rhodes, Mo...","[Andrew Obin, Andrew Kaplowitz, Scott Davis, J...",[Mike Roman],None,[Mike Roman]
2,3M Company (MMM) Q1 2023 Earnings Call Transcript,https://seekingalpha.com/article/4596435-3m-co...,2023-04-25,MMM,3M Company (NYSE:MMM) Q1 2023 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator\nLadies and gentlemen, thank you for ...",Operator\n[Operator Instructions] Our first qu...,"[Bruce Jermeland, Mike Roman, Monish Patolawala]","[Andrew Kaplowitz, Stephen Tusa, Scott Davis, ...",[Mike Roman],None,[Mike Roman]
3,3M Company (MMM) Q4 2022 Earnings Call Transcript,https://seekingalpha.com/article/4571946-3m-co...,2023-01-24,MMM,3M Company (NYSE:MMM) Q4 2022 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator\nLadies and gentlemen, thank you for ...",Operator\n[Operator Instructions] Our first qu...,"[Bruce Jermeland, Mike Roman, Monish Patolawala]","[Scott Davis, Andrew Kaplowitz, Andrew Obin, S...",[Mike Roman],None,[Mike Roman]
4,3M Company (MMM) Q3 2022 Earnings Call Transcript,https://seekingalpha.com/article/4548991-3m-co...,2022-10-25,MMM,3M Company (NYSE:MMM) Q3 2022 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator\nLadies and gentlemen, thank you for ...",Operator\n[Operator Instructions] Our first qu...,"[Bruce Jermeland, Mike Roman, Monish Patolawala]","[Scott Davis, Andrew Obin, Joe Ritchie, Chris ...",[Mike Roman],None,[Mike Roman]
5,3M Company (MMM) CEO Mike Roman on Q2 2022 Res...,https://seekingalpha.com/article/4525872-3m-co...,2022-07-26,MMM,3M Company (NYSE:MMM) Q2 2022 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator\nLadies and gentlemen, thank you for ...",Operator\n[Operator Instructions] Our first qu...,"[Bruce Jermeland, Mike Roman, Monish Patolawal...","[Andrew Obin, Scott Davis, Andrew Kaplowitz, S...",[Mike Roman],None,[Mike Roman]
6,3M Company (MMM) CEO Mike Roman on Q1 2022 Res...,https://seekingalpha.com/article/4503593-3m-co...,2022-04-26,MMM,3M Company (NYSE:MMM) Q1 2022 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator\nLadies and gentlemen, thank you for ...",Operator\n[Operator Instructions] Our first qu...,"[Bruce Jermeland, Mike Roman, John Banovetz, M...","[Steve Tusa, Scott Davis, Julian Mitchell, Jef...",[Mike Roman],None,[Mike Roman]
7,3M Company (MMM) CEO Mike Roman on Q4 2021 Res...,https://seekingalpha.com/article/4481561-3m-co...,2022-01-25,MMM,3M Company (NYSE:MMM) Q4 2021 Earnings Confere...,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,"Operator\nLadies and gentlemen, thank you for ...",Operator\n[Operator Instruction

In [38]:
df_sandp500_ECT_merge_without_spilt.to_csv("D:/Script/sandp500_conference_earning_transcript_participants.csv", index=False)

In [ ]:
# Since file is too huge, filter with only Presentation', 'QandA', 'CompanyParticipants', 'CallParticipants', 'title' part
df_filtered = df_sandp500_ECT_merge_without_spilt[['Presentation', 'QandA', 'CompanyParticipants', 'CallParticipants', 'title']]
df_filtered.to_csv("D:/Script/sandp500_pre_qanda_participantstranscript.csv", index=False)

In [37]:
df_sandp500_ECT_merge_without_spilt['Presentation'][1]

"Operator\nLadies and gentlemen, thank you for standing by. Welcome to the 3M Second Quarter Earnings Conference Call. [Operator Instructions] As a reminder, this conference is being recorded, Tuesday, July 25, 2023.\nI would now like to turn the call over to Bruce Jermeland, Senior Vice President of Investor Relations at 3M.\nBruce Jermeland\nThank you and good morning, everyone and welcome to our second quarter earnings conference call. With me today are Mike Roman, 3M's Chairman and Chief Executive Officer; Monish Patolawala, our Chief Financial and Transformation Officer; and Kevin Rhodes, our Chief Legal Officer. Mike, Kevin and Monish will make some formal comments, then we will take your questions. Please note that today's earnings release and slide presentation accompanying this call are posted on the homepage of our Investor Relations website at 3m.com.\nPlease turn to Slide 2. Please take a moment to read the forward-looking statement. During today's conference call, we'll be